In [1]:
import psycopg2
import geoplotlib
from geoplotlib.utils import BoundingBox, DataAccessObject
from geoplotlib.colors import ColorMap
from pandas import DataFrame
import json
import pandas as pd

# Replace with your database credentials
db_connection = psycopg2.connect(
    user="tester",
    password="1234",
    host="localhost",
    port="5432",
    database="ThesisTry1"
)

#updated query to add anzahlmodule to extract Q
query = """
SELECT 
    we."EinheitMastrNummer", we."Landkreis",
    SUBSTRING(we."Gemeindeschluessel" FROM 1 FOR 2) AS Kreisnummer,
    eeg."InstallierteLeistung"
FROM 
    wind_extended As we, wind_eeg as eeg
WHERE
    SUBSTRING(CAST(we."Registrierungsdatum" AS TEXT) FROM 1 FOR 4) = '2022'
GROUP BY 
    we."EinheitMastrNummer", we."Landkreis", SUBSTRING(we."Gemeindeschluessel" FROM 1 FOR 2), eeg."InstallierteLeistung";
"""

# Create a cursor and execute the query
cursor = db_connection.cursor()
cursor.execute(query)

# Fetch all the rows from the result
results = cursor.fetchall()

kreisliste = pd.DataFrame(results)
kreisliste.columns = ['EinheitMastrNummer','landkreis','kreisnummer','InstallierteLeistung']

# Drop rows with missing longitude or latitude values
kreisliste = kreisliste.dropna()


In [2]:
#Without_outliers
# Create a new DataFrame containing rows where 'q' is not between 300 and 500
filtered_df = kreisliste[(kreisliste['InstallierteLeistung'] < 20000) & (kreisliste['InstallierteLeistung'] > 100)].copy()
display(filtered_df)
#display(kreisliste)

,EinheitMastrNummer,landkreis,kreisnummer,InstallierteLeistung
0,SEE940880900352,Hamm,05,600.0
1,SEE984512830174,Helmstedt,03,660.0
5,SEE995891233685,Schleswig-Flensburg,01,660.0
6,SEE992670413870,Salzlandkreis,15,2500.0
7,SEE923187787123,Göttingen,03,850.0
...,...,...,...,...
404481,SEE902807354952,Nürnberg,09,3336.0
404483,SEE976004485702,Dithmarschen,01,3199.0
404485,SEE997322419743,Euskirchen,05,720.0
404486,SEE932108094950,Dithmarschen,01,5600.0
